In [41]:
import pandas as pd


df = pd.read_csv("../train_data/links.csv")
df.dropna(subset=["title", "content"], inplace=True, how="all")
df = df.fillna('')
df["title_content"] = df["title"] + df["content"]

In [42]:
%%time

import langdetect
from multiprocessing import Pool, cpu_count
from whatthelang import WhatTheLang

wtl = WhatTheLang()

def detect_lang(text):
    try:
#         return langdetect.detect(text[0:50])
        return wtl.predict_lang(text)
    except:
        return None

# with Pool(cpu_count()) as p:
#     df["lang"] = p.map(detect_lang, df["title_content"])
df["lang"] = [ detect_lang(c) for c in df["title_content"]]

CPU times: user 2.14 s, sys: 120 ms, total: 2.26 s
Wall time: 2.44 s


In [45]:
print(df[["title", "lang"]].groupby(['lang']).agg(['count']))

             title
             count
lang              
CANT_PREDICT    19
ar               5
bg               3
ca               9
ceb              1
cs              33
da               4
de              37
el              12
en            3255
eo               2
es             360
fa               2
fi               2
fr              58
ga               1
gl              36
hi               1
hr               3
hu              14
hy               1
id               1
it              65
ja               3
jbo              1
ka               3
ko               1
la               8
lb               2
lt               1
mk               1
mr               1
nds              1
nl               7
nn               1
no               2
oc               1
pl              23
pt            1691
ro              72
ru               7
sk               3
sl               1
sq               3
sv              22
te               1
tl               7
tr               1
uk               2
ur          

In [111]:
df_ = df[df["lang"] == 'en'][0:500].append(df[df["lang"] == 'es'][0:500]).append(df[df["lang"] == 'pt'][0:500])
texts = df_["title_content"].values.tolist()
print(len(texts))
texts[1]

1366


'Thank YouThank you for installing our TV for Google Chrome™, you can start using the extension free of charge by clicking on it.\n\nHelp to keep us free, try this great FREE offer:'

In [112]:
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import unidecode

all_stopwords = list(map(unidecode.unidecode,
            stopwords.words('english') + \
            stopwords.words('portuguese') + \
            stopwords.words('spanish')))

text = """
ALERTA GERAL Não beba Skol! A propaganda parou… Por quê? Reparem… A propaganda quase não se vê mais na mídia… Porque será??? Estamos repassando o e-mail abaixo para conhecimento e prevenção, principalmente para aqueles que bebem esta cerveja: Skol. Este e-mail está sendo repassado dentro do Hospital que trabalha uma pessoa amiga. Fato já está confirmado:Vinte e três pessoas já passaram pelo Hospital das Clínicas com um mesmo sintoma: falta de atividade renal e o aparecimento de tumores no reto. Todos os internados relataram o começo das dores e a conseqüente internação após ingerirem altas doses de Skol. Pesquisas realizadas pelo renomado Instituto Fleury, apontaram grande quantidade de Fenofinol, Almeido e Voliteral, substâncias tóxicas e que causam, respectivamente, a má atividade dos rins e câncer. Segundo Dr. Paulo José Teixeira, formado pela USP e Especialista em Toxicologia, as pessoas não devem ingerir mais a citada cerveja. A Direção da AMBEV já assumiu sua culpa e prometeu indenizar os pacientes e todos aqueles que venham a se contaminar com a cerveja. Pelo amor de “DEUS”… Passem esta mensagem para frente. Pelo sim e pelo não, vamos tentar remediar enquanto há tempo. Lembre-se: Divulgar a todos de sua família, é a consciência de cada um que deve decidir, mas a nossa deve estar tranqüila. Monique Freitas Soc.Bras.de Cardiologia/Secretaria. Por favor não deixem de repassar esta mensagem para todos que vocês conhecem. Repassando
"""

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfidf = TfidfVectorizer(
            strip_accents='ascii',
            ngram_range=(1, 3),
            max_df=0.05,
            min_df=3,
            use_idf=True,
            lowercase=True,
            stop_words=all_stopwords)

tfidf.fit_transform(texts)

<1366x22574 sparse matrix of type '<class 'numpy.float64'>'
	with 191793 stored elements in Compressed Sparse Row format>

In [114]:
AMOUNT_OF_KEYWORDS = 8

def tfidf_keywords(text):
    response = tfidf.transform([text])
    feature_array = np.array(tfidf.get_feature_names())

    response_arr = response.toarray()
    valid_words = [word for word in response_arr[0] if word > 0.0]

    if len(valid_words) < AMOUNT_OF_KEYWORDS:
        return []

    tfidf_sorting = np.argsort(response_arr).flatten()[::-1]

    return feature_array[tfidf_sorting][:AMOUNT_OF_KEYWORDS].tolist()

" ".join(tfidf_keywords(text))

'cerveja atividade mail mensagem propaganda hospital deve bebem'

In [115]:
text2 = """
 Um projeto em tramitação no Senado Federal pretende obrigar que empresas responsáveis por sites e aplicativos de redes sociais garantam a autorização do usuário antes que ele seja incluído em um novo grupo ou evento. O texto altera o Marco Civil da Internet e prevê advertência e multa em caso de descumprimento. 

A norma seria aplicada, por exemplo, a contas e páginas no Facebook, WhatsApp e Telegram. 

O PLS (projeto de lei do Senado) 347/2016, de autoria da senadora Vanessa Grazziotin (PCdoB-AM), foi aprovado no último dia 7 pela CCJ (Comissão de Constituição, Justiça e Cidadania) e agora segue para votação final na CCT (Comissão de Ciência, Tecnologia, Inovação, Comunicação e Informática).

No texto, a senadora diz que a mudança é necessária porque "há redes sociais que possibilitam aos membros a inclusão de outros usuários em grupos, páginas, comunidades e similares, sem a autorização prévia do incluído - ambientes estes muitas vezes com conteúdo agressivo, pornográfico, ou simplesmente desinteressante ao usuário adicionado, que em momento algum foi questionado sobre o interesse em participar de tais espaços".
Veja também:

    Acabou a festa: Instagram agora denuncia captura de tela no Stories
    Carro mais velho liberado: SP confirma mudança nas regras para Uber e afins
    Facebook admite que falhou em unir as pessoas e testa botão de "não gostei"

Apesar da aparente boa intenção da proposta, especialistas ouvidos pelo UOL dizem que o projeto não é necessário, já que é possível sair voluntariamente desses ambientes indesejados, e ainda poderia criar novas dificuldades para empresas e internautas. 

"Hoje já existem ferramentas que possibilitam ao usuário sair do grupo, denunciar spam, dizer que não quer receber mais convites de determinadas páginas ou pessoas", afirma o advogado Rony Vainzof. "Pode parecer interessante a ideia de ter que receber um convite prévio para participar de um grupo, mas isso seria aplicável na prática? O que teria que constar nesse consentimento?", questiona.  

No texto do projeto, uma das justificativas para a alteração é evitar que empresas utilizem dados dos usuários sem autorização. Isso porque, segundo o projeto, algumas redes sociais, de forma não autorizada, "acessam a lista de contatos de seus membros para atrair novos usuários, convidando integrantes dessa lista, em nome dos respectivos membros, a ingressarem na rede". 

Para Carlos Affonso Souza, diretor do ITS Rio (Instituto de Tecnologia e Sociedade do Rio de Janeiro), o projeto não ataca o problema e desvia o foco da discussão. "A Casa deveria estar concentrada na aprovação da lei de proteção de dados pessoais [PLS 330/2013], que torna mais clara a proibição da venda de dados, tanto na internet quanto fora dela", diz. 

É uma escolha de prioridades desafortunada que os nossos senadores passem um tempo discutindo o regime de convites quando poderiam estar discutindo lei de dados no Brasil" Carlos Affonso Souza

O PLS 330/2013 está na CAE (Comissão de Assuntos Econômicos) aguardando parecer do relator, o senador Sérgio de Castro (PDT-ES). 

Responsabilidade e dano moral  

Na CCJ, o projeto da senadora Vanessa Grazziotin recebeu duas emendas do relator Romero Jucá (PMDB-RR). 

A primeira mudança retira do texto a responsabilidade do usuário que adicionou outro sem autorização. Para Jucá, "a culpa pela ausência de mecanismos que asseguram a coleta do consentimento prévio é da aplicação, e não dos usuários". 

A segunda emenda diz respeito à garantia de dano moral em caso de inclusão do usuário sem consentimento prévio. "Hoje é discutível se existe dano moral em casos como esse e, se existir dano, talvez não seja indenizável. Isso porque os tribunais brasileiros já decidiram que 'mero aborrecimento não gera dano moral' e esse caso é o típico caso de mero aborrecimento. Então, o que o senador [Romero Jucá] fez com a mudança do texto foi tentar forçar que o Judiciário reconheça o dano", diz Souza. 

"Isso não quer dizer que um dano nesses ambientes não possa acontecer. Você pode ser inserido em um grupo e se sentir ofendido por algo, mas hoje é possível mover uma ação indenizatória contra a pessoa que te incluiu. Para isso, não é preciso obrigar as aplicações a desenvolverem um sistema que burocratiza todos os grupos só para resolver o caso de 0,0001%", afirma o diretor do ITS Rio.
"""
" ".join(tfidf_keywords(text2))

'dano projeto usuario autorizacao texto moral ambientes usuarios'

In [19]:
tfidf_keywords("foo bar baz texto")

[]